In [17]:
import sys
sys.path.append('../experiments')
import matplotlib.pyplot as plt
import numpy as np

In [47]:
from __future__ import print_function

import warnings

from deepiv.models import Treatment, Response
import deepiv.architectures as architectures
import deepiv.densities as densities

import tensorflow as tf

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.merge import Concatenate

import numpy

import data_generator
import pickle

n = 5000
dropout_rate = min(1000./(1000. + n), 0.5)
epochs = int(1500000./float(n)) # heuristic to select number of epochs
epochs = 300
batch_size = 100
images = False

def datafunction(n, s, images=images, test=False):
    return data_generator.demand(n=n, seed=s, ypcor=0.5, use_images=images, test=test)

x, z, t, y, g_true = datafunction(n, 1)
data = {'x': x, 'z': z, 't': t, 'y': y}


print("Data shapes:\n\
Features:{x},\n\
Instruments:{z},\n\
Treament:{t},\n\
Response:{y}".format(**{'x':x.shape, 'z':z.shape,
                        't':t.shape, 'y':y.shape}))

path_to_store_data = "../saved_data/demand.pkl"
with open(path_to_store_data, 'wb') as handle:
    pickle.dump(data, handle)
    


Data shapes:
Features:(5000, 8),
Instruments:(5000, 1),
Treament:(5000, 1),
Response:(5000, 1)


/Users/amehrjou/Projects/DualIV/Codes/python/DeepIV/env_deepiv/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)


In [33]:
# Build and fit treatment model
instruments = Input(shape=(z.shape[1],))
x = Dense(64, activation='relu')(instruments)
x = Dense(64, activation='relu')(x)
#est_treat = Dense(1)(x)
est_treat = densities.mixture_of_gaussian_output(x, 10)

In [27]:
treatment_model = Treatment(inputs=[instruments], outputs=est_treat)
treatment_model.compile('adam',
                        loss="mixture_of_gaussians",
                        n_components=10)



Instructions for updating:
Use `tf.random.categorical` instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [29]:
treatment_model.fit([z],t, epochs=epochs)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/100
1000/1000 [==============================] - 0s - loss: 1.4801     
Epoch 2/100
1000/1000 [==============================] - 0s - loss: 1.4713     
Epoch 3/100
1000/1000 [==============================] - 0s - loss: 1.4688     
Epoch 4/100
1000/1000 [==============================] - 0s - loss: 1.4641     
Epoch 5/100
1000/1000 [==============================] - 0s - loss: 1.4617     
Epoch 6/100
1000/1000 [==============================] - 0s - loss: 1.4595     
Epoch 7/100
1000/1000 [==============================] - 0s - loss: 1.4578     
Epoch 8/100
1000/1000 [==============================] - 0s - loss: 1.4581     
Epoch 9/100
1000/1000 [==============================] - 0s - loss: 1.4570     
Epoch 10/100
1000/1000 [==============================] - 0s - loss:

1000/1000 [==============================] - 0s - loss: 1.4568     
Epoch 93/100
1000/1000 [==============================] - 0s - loss: 1.4564     
Epoch 94/100
1000/1000 [==============================] - 0s - loss: 1.4569     
Epoch 95/100
1000/1000 [==============================] - 0s - loss: 1.4564     
Epoch 96/100
1000/1000 [==============================] - 0s - loss: 1.4565     
Epoch 97/100
1000/1000 [==============================] - 0s - loss: 1.4562     
Epoch 98/100
1000/1000 [==============================] - 0s - loss: 1.4565     
Epoch 99/100
1000/1000 [==============================] - 0s - loss: 1.4566     
Epoch 100/100
1000/1000 [==============================] - 0s - loss: 1.4580     


In [30]:
treatment_model